# Veri Seti ve Preprocessing

Veri Seti işlemleri ve kütüphaneler

In [2]:
!pip install xlrd


In [3]:
import nltk
import re #regex metin düzenleme işlemleri için
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords #dillerde anlamsız kelimeleri filtrelemek için
from sklearn.feature_extraction.text import TfidfVectorizer as tfidV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV

In [4]:
ds = pd.read_excel('Final_Dataset.xlsx')
ds[['content', 'category']].head() # Veri Kontrolü

,content,category
0,"Irak Petrol Bakanı Abdul Kerim Luaibi, Ankara...",Dunya
1,"Fransız güçleri, El Kaide'nin Kuzey Afrika'dak...",Dunya
2,"Wikileaks'in kaynağı, Amerika'da hakim karşısı...",Dunya
3,"İsrail'in, 6 Fetih Hareketi yöneticisini gözal...",Dunya
4,Myanmar'ın kuzeyinde bugün meydana gelen depre...,Dunya


natural Language ToolKit StopWords Kurulumu

In [5]:
nltk.download('stopwords') # filtre indiriliyor ve türkçe için ayarlanıyor.
lemmatizer = WordNetLemmatizer()
stop_words_tr = set(stopwords.words('turkish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
ds["category"].value_counts()

,count
category,
Spor,10738
Magazin,9628
Guncel,7146
Dunya,5850
Ekonomi,4404
Siyaset,2925
Saglik,2402
Teknoloji,2070
Kultur,1833


In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

PreProcessing işlemleri

In [8]:
def clean_text(text):
    text = text.lower()
    # noktalama kaldırma, rakamları kaldırma ve kelimelere bölme
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    tokens = text.split()
    #tokens içerisindeki kelimeleri stop_word olanlarını kaldırıyor.
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words_tr]
    return ' '.join(tokens)

In [9]:
ds = ds.dropna(subset=['content', 'category'])  # NAN Boş verileri temizleme
ds['clean_content'] = ds['content'].apply(clean_text) #clean text fonksiyonu kullanılıyor

In [10]:
ds[['clean_content','category']].head(10)

,clean_content,category
0,irak petrol bakanı abdul kerim luaibi ankaranı...,Dunya
1,fransız güçleri el kaidenin kuzey afrikadaki a...,Dunya
2,wikileaksin kaynağı amerikada hakim karşısına ...,Dunya
3,israilin fetih hareketi yöneticisini gözaltına...,Dunya
4,myanmarın kuzeyinde bugün meydana gelen deprem...,Dunya
5,somalinin başkenti mogadişunun doğusunda bulun...,Dunya
6,ilk uzay turisti milyarder işadamı dennis tito...,Dunya
7,yılında mekong nehri üzerinde çinli denizciyi ...,Dunya
8,japonyanın teknoloji devlerinden fujitsu akıll...,Dunya
9,rusya federal askeriteknik işbirliği servisi m...,Dunya


PreProcessing'in son aşaması vektörleştirme (kelimelerden Sayısal Değer Üretme )

In [36]:
vectorizer = tfidV(max_features=8000, ngram_range=(1,2), max_df=0.8, min_df=5)
X = vectorizer.fit_transform(ds['clean_content'])
y = ds['category']

In [38]:
y.unique()

array(['Dunya', 'Ekonomi', 'Guncel', 'Kultur', 'Magazin', 'Saglik',
       'Siyaset', 'Spor', 'Teknoloji', 'Yasam'], dtype=object)

# Model Eğitimi ve yapılandırılması

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [42]:
print(X_train.shape)

(38592, 8000)


In [32]:
params = {'alpha': [0.04, 0.03, 0.02, 0.01]}
grid = GridSearchCV(MultinomialNB(), param_grid=params, cv=5, scoring='f1_macro')
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': [0.04, 0.03, 0.02, 0.01]},
             scoring='f1_macro')

In [33]:
print("NB Model En iyi alpha:", grid.best_params_)
print("F1 skoru:", grid.best_score_)

NB Model En iyi alpha: {'alpha': 0.02}
F1 skoru: 0.6620845354215222


In [34]:
# Naive Bayes modeli
nb = MultinomialNB(alpha=0.02)
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)



In [35]:
print("🔹 Naive Bayes Sonuçları:")
print("Doğruluk:", accuracy_score(y_test, nb_pred))
print(confusion_matrix(y_test, nb_pred))
print(classification_report(y_test, nb_pred))


🔹 Naive Bayes Sonuçları:
Doğruluk: 0.6986940298507462
[[ 848   37   71    3  137   10   40   12   18    8]
 [  34  696   27    0  100    3    9   10   14    0]
 [  59   39  809    8  171   24  148   38   16   57]
 [   7    5   15  253   77    6    3    8    4    0]
 [ 159  216  244   40  955   41   89  135   31   11]
 [   2   15   19    1   28  432    5    4    3    1]
 [  46   12   94    2   41    3  358    2    4    0]
 [  10   14   19    2  102    5   10 1992    3    1]
 [  11   20    5    3   34   11    1    4  328    0]
 [  21    3  119    0   37    2    0    0    4   70]]
              precision    recall  f1-score   support

       Dunya       0.71      0.72      0.71      1184
     Ekonomi       0.66      0.78      0.71       893
      Guncel       0.57      0.59      0.58      1369
      Kultur       0.81      0.67      0.73       378
     Magazin       0.57      0.50      0.53      1921
      Saglik       0.80      0.85      0.83       510
     Siyaset       0.54      0.64   

In [23]:
param_grid = {
    'C': [2.5, 2.6, 2.7],
    'solver': ['liblinear', 'lbfgs']
}

grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid.fit(X_train, y_train)
print("LR model best param: ",grid.best_params_)
print("Best score:", grid.best_score_)

LR model best param:  {'C': 2.7, 'solver': 'liblinear'}
Best score: 0.6902986109565447


In [24]:
# Logistic Regression modeli
lr = LogisticRegression(max_iter=1000, C=2.7)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

In [39]:
print("\n🔹 Logistic Regression Sonuçları:")
print("Doğruluk:", accuracy_score(y_test, lr_pred))
print(confusion_matrix(y_test, lr_pred))
print(classification_report(y_test, lr_pred))


🔹 Logistic Regression Sonuçları:
Doğruluk: 0.7009742951907131
[[ 877   27   70    1  145    7   22   16   16    3]
 [  30  631   35    0  162    7    6   12   10    0]
 [  46   28  874    5  255   18   79   38   10   16]
 [  14    5   13  231  103    5    1    3    3    0]
 [ 158  181  248   34 1047   32   55  139   23    4]
 [   8   10   33    0   42  410    2    2    3    0]
 [  43   10  139    0   66    3  293    4    4    0]
 [   8    6   15    0   90    1    3 2032    3    0]
 [  14   15   12    4   54    3    0    7  308    0]
 [  22    2  118    0   50    2    0    0    2   60]]
              precision    recall  f1-score   support

       Dunya       0.72      0.74      0.73      1184
     Ekonomi       0.69      0.71      0.70       893
      Guncel       0.56      0.64      0.60      1369
      Kultur       0.84      0.61      0.71       378
     Magazin       0.52      0.55      0.53      1921
      Saglik       0.84      0.80      0.82       510
     Siyaset       0.64    

In [46]:
from sklearn.svm import LinearSVC

param_grid = {
    'C': [0.19, 0.18, 0.2, 0.21, 0.22,0.23],
}

# Model
svm = LinearSVC()

# Grid Search
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# En iyi parametreler
print("En iyi parametreler:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

En iyi parametreler: {'C': 0.2}
Best score: 0.6970098115589695


In [59]:

svm = LinearSVC(C=0.2,max_iter=100)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

In [60]:
print("🔹 LinearSVC Sonuçları:")
print("Doğruluk:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

🔹 LinearSVC Sonuçları:
Doğruluk: 0.7101990049751243
[[ 909   26   66    2  111   10   18   22   18    2]
 [  35  678   33    1  107    7    7   12   13    0]
 [  61   33  882    8  196   22   94   45   14   14]
 [  13    7   15  237   91    6    0    4    5    0]
 [ 180  197  246   36  967   35   73  155   28    4]
 [   6   12   26    0   24  434    2    2    4    0]
 [  45   12  126    1   46    3  318    8    3    0]
 [   9    8   12    2   51    1    3 2070    2    0]
 [  11   14   12    5   51    7    0    4  313    0]
 [  27    2  130    1   45    2    0    1    4   44]]
              precision    recall  f1-score   support

       Dunya       0.70      0.77      0.73      1184
     Ekonomi       0.69      0.76      0.72       893
      Guncel       0.57      0.64      0.60      1369
      Kultur       0.81      0.63      0.71       378
     Magazin       0.57      0.50      0.54      1921
      Saglik       0.82      0.85      0.84       510
     Siyaset       0.62      0.57     

In [15]:
sum(ds['category'].value_counts())
ds["category"].unique()


array(['Dunya', 'Ekonomi', 'Guncel', 'Kultur', 'Magazin', 'Saglik',
       'Siyaset', 'Spor', 'Teknoloji', 'Yasam'], dtype=object)